In [2]:
def get_review(hotel_code):
    num = 0
    max_num = 10
    review_list = []

    while num <= max_num:
        url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/reviews/HOTEL/{hotel_code}/contents?page={num}'
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        }
        res = requests.get(url,headers=headers)
        review = json.loads(res.text)

        for con in review['data']['contents']: 

            review_list.append({
                'reviewAt' : con['reviewAt'],
                'nickname' : con['nickname'],
                'comment' : con['comment'],
                'avgScore' : con['avgScore'],
                'avgScoreDesc' : con['avgScoreDesc'],
                'itemDetailName' : con['itemDetailName'],
                'helpfulCount' : con['helpfulCount'],
                'mainImagePath' : con['mainImagePath']
                                })
        

        # 페이지 추가
        max_num = review['data']['totalElements'] // 20
        num += 1

    return pd.DataFrame(review_list)



In [3]:
import requests
import json
import pandas as pd
from datetime import datetime

def check(search):

    today = datetime.now().strftime('%Y-%m-%d')
    url = 'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/search/srp/list'

    data = {
        "serviceType": "HOTEL",
        "itineraries": {"stayDate": f"{today}", "stays": "1"},
        "query": {"term": f"{search}", "type": "term"},
        "page": 0
    }    
    data_json = json.dumps(data).encode('utf-8')

    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'Content-Type': 'application/json'}

    res = requests.post(url, headers=headers, data=data_json)
    search_list = json.loads(res.text)

    max_num = min(5,search_list['data']['totalElements'])
    card_list = []

    for card in search_list['data']['cards'][:max_num]:
        card_list.append({
            'idx' : card['data']['idx'],
            'desc' : card['data']['desc']['title'],
            'prf' : card['data']['badge']['pfr'],
            'review' : card['data']['badge']['review'],
            'lat' : card['data']['location']['lat'],
            'lon' : card['data']['location']['lon']
        })    

    question = []

    for idx,i in enumerate(card_list):
        question.append({
            '번호' : idx+1,
            '숙소명': i['desc'],
            '리뷰' : i['review']
        })
    question.append('>>>>>>>>>>>>>>>> 해당하는 결과의 번호만 입력하세요')    

    idx = int(input(question)) -1

    hotel_code = card_list[idx]['idx']

    return get_review(hotel_code)

    

In [6]:
check('하얏트')

,reviewAt,nickname,comment,avgScore,avgScoreDesc,itemDetailName,helpfulCount,mainImagePath
0,2023.08.26,k01**,뷰도 너무 좋고 무엇보다 부대시설이 최고에요~!! 수영당도 너무 좋구요~! 엘리베이...,5.0,최고,스탠다드 → 프리미엄 남산전망 킹 / 더블 침대 1개,1,[https://img.dailyhotel.me/review/HOTEL/81456/...
1,2023.11.02,als**,이번에도 모든면에서 만족스러웠습니다 단점은 너무나 높은 언덕위에 위치해서 산책하고 ...,4.8,최고,프리미엄 남산전망 킹,0,[https://img.dailyhotel.me/review/HOTEL/81456/...
2,2023.09.11,man**,식사랑 수영장 좋아요\n비데가 없어서 아쉽고\n청소상태 아쉬워요(리모콘....보고깜...,4.3,매우좋음,스탠다드 킹,0,[]
3,2023.07.04,jht**,시설도 조식도 모두 좋았어요 \n단지 화장실이 비데가 아닌것은 옥의티\n저렴하게 잘...,4.5,최고,스탠다드 킹 / 더블 침대 1개,0,[]
4,2023.06.09,jju**,"객실이 너무 깔끔했고, 수영장, 조식 시 직원들 서비스 모두 너무 만족스러웠습니다....",5.0,최고,[하얏트 글로벌 객실 특가 2인 조식 포함]프리미엄 한강전망 (1 킹) / 킹 1개,2,[https://img.dailyhotel.me/review/HOTEL/23526/...
...,...,...,...,...,...,...,...,...
1624,2018.09.15,seu**,욕실에 곰팡이가 많아요ㅠㅠ 넘 돈 아까워,1.8,,[2인 조식] 스탠다드 (1 킹) 더블1,0,[]
1625,2018.03.13,son**,"방이 너무 건조하고 먼지가 많이 쌓여있었어요,",2.0,,[21시 체크인 - Room Only] High Floor 더블,0,[]
1626,2018.01.12,act**,야경이 아주 예뻤어요,2.0,,[21시 체크인 - Room Only] Grand Room 더블,0,[]
1627,2017.08.10,jiy**,서비스나 음식. 다별로예요,1.0,,[Poolside BBQ PKG] Grand Room 싱글 + 싱글,0,[]
